In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
def defp(pt):
    try:    sys.path.index(pt)
    except: sys.path.append(pt)
#defp('/ml.dev/bin')

from qoreliquid import *
from bitmex import *

import numpy as n
import qgrid as qg # https://github.com/quantopian/qgrid#installation

%pylab inline
from pylab import rcParams
rcParams['figure.figsize'] = 20, 5
import seaborn as sns
sns.set()

p.set_option('display.max_rows', 1000)

pm = PortfolioModeler()
cmc = CoinMarketCap()

In [ ]:
#rng = range(-10,10)
rng = n.linspace(-10,10,100)
x = n.array(rng)
y = x**2 #power(x,2)
"""
dydx = dydx(y,x)
#print x.shape
#print y.shape
#print dydx.shape
#dydx = dydxSym('x**2',x)
#print dydx
plot(x, dydx)
plt.show()
"""

In [ ]:
%load_ext line_profiler
#@profile
def rere():
    import pymongo as mong
    mon = mong.MongoClient()
    mdf = p.DataFrame()
    for i in mon.ql.coins.find():
        df = p.DataFrame(i)
        #df = df.head(10)
        res = list(df.loc[:,'data'.split(' ')]['data'])
        dfi = p.DataFrame(res)
        dfi['q'] = df.index
        dfi = dfi.set_index('q')
        dfi['time'] = df['time']
        dfi['symbolTS'] = map(lambda x: '%s-%s' % (dfi.loc[x, 'symbol'], dfi.loc[x, 'time']), dfi.index)
        dfi = dfi.set_index('symbolTS')
        #print df
        #with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000, 'display.max_colwidth', -1):
        #    #print dfi
        #    #print dfi.shape
        mdf = mdf.combine_first(dfi)
        #break
    with p.option_context('display.max_rows', 400, 'display.max_columns', 4000, 'display.width', 1000000, 'display.max_colwidth', -1):
        #print mdf
        print mdf.shape
%lprun -f rere rere()

In [ ]:
#df = p.read_csv('/tmp/dfi.csv', index_col=[0])
#' '.join(list(df.columns))
#
#df = p.read_csv('/tmp/dfi.csv.0', index_col=[0], header=None)
print df.shape
#df = df.rename(dict(zip(range(1,11), 'marketCap name pcnt1h pcnt24h pcnt7d price symbol volume volumePerMarketcap time'.split(' '))), axis='columns')
print df.columns
df['indx'] = df.index

"""
#with p.option_context('display.max_rows', 400, 'display.max_columns', 4000, 'display.width', 1000000, 'display.max_colwidth', -1):
dfp = df.tail(250).pivot('indx', 'symbol', 'volume')
dfp = dfp.ffill().bfill()
dfp = normalizeme(dfp)
#dfp = sigmoidme(dfp)
#print dfp
dfp.plot()
"""


In [ ]:
mdf = df[df['symbol'] == 'ZRX'].loc[:, 'marketCap price volume time'.split(' ')]#.set_index('time')#.tail(10000)
mdf = normalizeme(mdf)
#mdf = sigmoidme(mdf)
mdf.plot()
plt.show()
#mdf
res = dydx(mdf['volume'], mdf['time'])
res = p.DataFrame(res, index=mdf.index, columns=['volume'])
#print res
res = normalizeme(res)
res = sigmoidme(res)
res.plot()
#res.hist(bins=100)
#plt.scatter(res.index, res['volume'])
plt.show()

In [ ]:
#nmdf = mdf.get_values()
#p.DataFrame(nmdf, columns=mdf.columns, index=mdf.index)
mdf0 = mdf
#mdf0 = mdf0[(mdf['symbol'] == 'BTC') | (mdf['symbol'] == 'ETH')]
#print mdf0.head(5)
val = 'volume'
val = 'marketCap'
mdf0 = mdf0.loc[:,('time symbol %s'%val).split()]#.plot()#.get_values()
mdf0 = mdf0[mdf0.index > 1.512845e+09]
print mdf0.shape
mdf0 = mdf0.drop_duplicates().tail(40000)
mdf0 = mdf0.pivot('time', 'symbol',val)
#mdf0 = normalizeme(mdf0)
#mdf0 = sigmoidme(mdf0)
#mdf0.fillna(0).plot()
#mdf0 = mdf0.sort_values(by'')
#print mdf0
fig, ax = plt.subplots(figsize=(30,40))         # Sample figsize in inches
#sns.heatmap(mdf0.transpose())
#sns.heatmap(mdf0, center=0.5, annot=True, linewidths=0, ax=ax, cmap="YlGnBu")
#mdf0#.corr()
#desc(mdf0)
mdf0

In [ ]:
#print y
mmo = mdf0
for i in mmo.columns:
    y = mmo.loc[:,i].get_values()
    x = n.array(list(mmo.index))
    res = dydx(y, x)
    mmo[i] = res
mmo = mmo.fillna(0)

df = mmo.get_values()
ind = (n.sum(df, 1) > 0)
#print df.shape
#print ind.shape
#print df
#print ind
df2 = df[ind, :]
df2.shape
df2

mmo = mmo.loc[ind, :]

#mmo = mmo.ffill()
mmo = normalizeme(mmo)
mmo = sigmoidme(mmo)
#mmo.plot()
#plt.show()
mmo

In [ ]:
#5000 * ([70 30*[20 80]/100]'/100)

#n.array([70, (n.matrix('20 80', dtype=n.float)/100).A])

In [ ]:
none = desc(df)
pp = sns.pairplot(df)
plt.show()
none

In [ ]:
#  all coins volumePerMarketcap / pcnt scatter plot matrix
from qoreliquid import normalizeme
to = cmc.getAllTokens(tokens=False)
ddd = to
ddd = pm.generatePortfolioT1Supply(ddd, balance=(4129.06), risk=1)
ddd = ddd.sort_values(by='volumePerMarketcap', ascending=False)
ddd = ddd.loc[:,'marketCap riskOn volumePerMarketcap vb'.split()]
ddd = normalizeme(ddd)
ddd = sigmoidme(ddd)
ddd.plot()
plt.show()
#plt.scatter(ddd['marketCap'], ddd['volumePerMarketcap'])
#plt.show()
#ddd = to.loc[:,'pcnt1h pcnt24h pcnt7d'.split(' ')].fillna(0)
ddd = ddd.loc[:,'marketCap riskOn volumePerMarketcap vb'.split(' ')].fillna(0)
ddd = normalizeme(ddd)
ddd = sigmoidme(ddd)
#print ddd
sns.pairplot(ddd, size=10)#, hue='pcnt1h')
plt.show()

none = desc(ddd)
#qg.show_grid(ddd)
qg.show_grid(ddd, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})
#cmc.tokensOnOtherExchanges('etherdelta')

In [ ]:
# All Ethereum Tokens PoC
tokens=True
dfbnb = cmc.getAllTokens(tokens=tokens)
#if tokens: dfbnb = dfbnb[dfbnb['quote'] == 'ETH']
#else:      dfbnb = dfbnb[dfbnb['quote'] == 'BTC']
li = list(dfbnb.index)
dft = cmc.getAllTokens(tokens=tokens)
#dft1s = dft[dft['marketCap'] <= 1e6]
dft1s = pm.generatePortfolioT1Supply(dft, balance=(4129.06), risk=1)
dft1s = dft1s.loc[li,:]
sortby = 'riskOn'
#sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
dft1s = dft1s.sort_values(by=sortby, ascending=False)
#none = desc(dft1s)
qg.show_grid(dft1s)
v = DataViz()
dft1s = v.visualizePortfolio(dft1s, li, figsize=300)
#dft1s.sort_values(by='balanceMarketcapPcnt', ascending=False).loc[:,'pcnt7d'].plot()
#dft1s

In [ ]:
# Poloniex PoC
b = Poloniex()
dfbnb = b.getCurrencies()
print dfbnb['quote']
tokens=True
if tokens: dfbnb = dfbnb[dfbnb['quote'] == 'ETH']
else:      dfbnb = dfbnb[dfbnb['quote'] == 'BTC']
li = list(dfbnb['base'])
dft = cmc.getAllTokens(tokens=tokens)
#dft1s = dft[dft['marketCap'] <= 1e6]
dft1s = pm.generatePortfolioT1Supply(dft, balance=(4129.06), risk=1)
dft1s = dft1s.loc[li,:]
sortby = 'riskOn'
#sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
dft1s = dft1s.sort_values(by=sortby, ascending=False)
#none = desc(dft1s)

v = DataViz()
dft1s = v.visualizePortfolio(dft1s, li, figsize=300)

qg.show_grid(dft1s)
#dft1s.sort_values(by='balanceMarketcapPcnt', ascending=False).loc[:,'pcnt7d'].plot()
#dft1s

In [ ]:
rm -f /mldev/bin/scraper_cache.sqlite

In [ ]:
dfbnb.loc['VRS',:]

In [ ]:
# Top 100 coinmarketcap 
#rmScraperCache()
tokens=False
cmc=CoinMarketCap()
dfbnb = cmc.getAllTokens(tokens=tokens)
#if tokens: dfbnb = dfbnb[dfbnb['quote'] == 'ETH']
#else:      dfbnb = dfbnb[dfbnb['quote'] == 'BTC']
li = list(dfbnb.index)
dft = cmc.getAllTokens(tokens=tokens)
#dft1s = dft[dft['marketCap'] <= 1e6]
dft1s = pm.generatePortfolioT1Supply(dft, balance=(4129.06), risk=1)
dft1s = dft1s.loc[li,:]
sortby = 'riskOn'
#sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
dft1s = dft1s.sort_values(by=sortby, ascending=False)
#none = desc(dft1s)
qg.show_grid(dft1s)
v = DataViz()
v.getAllTokens(tokens=tokens)
v.df = v.df.sort_values(by='marketCap', ascending=False).head(100)
threshold = 0.5
#dft1s = dft1s[(dft1s['vb'] > threshold) & (dft1s['volumePerMarketcap'] > threshold) & (dft1s['riskOn'] > threshold)].sort_values(by='vb', ascending=False)
#dft1s = v.visualizePortfolio(dft1s, li, figsize=300)
figsize=30
showClustermap=True
v.heatmap(maxx=1000e9, minn=0, usdt=False, figsize=figsize, sortby=sortby, threshold=threshold)
dfs = genDFP(list(v.pdf['id']), max=20, logy=yy, normalize=yy, sigmoid=yy, showPlot=False)
cm = clustermap(dfs, verbose=False, figsize=10, showClustermap=showClustermap)
#dft1s.sort_values(by='balanceMarketcapPcnt', ascending=False).loc[:,'pcnt7d'].plot()

In [ ]:
# Top 100 coinmarketcap 
#rmScraperCache()
tokens=False
cmc=CoinMarketCap()
dfbnb = cmc.getAllTokens(tokens=tokens)
#if tokens: dfbnb = dfbnb[dfbnb['quote'] == 'ETH']
#else:      dfbnb = dfbnb[dfbnb['quote'] == 'BTC']
li = list(dfbnb.index)
dft = cmc.getAllTokens(tokens=tokens)
#dft1s = dft[dft['marketCap'] <= 1e6]
dft1s = pm.generatePortfolioT1Supply(dft, balance=(4129.06), risk=1)
dft1s = dft1s.loc[li,:]
sortby = 'riskOn'
#sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
dft1s = dft1s.sort_values(by=sortby, ascending=False)
#none = desc(dft1s)
qg.show_grid(dft1s)
v = DataViz()
v.getAllTokens(tokens=tokens)
v.df = v.df.sort_values(by='marketCap', ascending=False).head(100)
threshold = 0.35
#dft1s = dft1s[(dft1s['vb'] > threshold) & (dft1s['volumePerMarketcap'] > threshold) & (dft1s['riskOn'] > threshold)].sort_values(by='vb', ascending=False)
#dft1s = v.visualizePortfolio(dft1s, li, figsize=300)
figsize=30
showClustermap=True
v.heatmap(maxx=1000e9, minn=0, usdt=False, figsize=figsize, sortby=sortby, threshold=threshold)
dfs = genDFP(list(v.pdf['id']), max=20, logy=yy, normalize=yy, sigmoid=yy, showPlot=False)
cm = clustermap(dfs, verbose=False, figsize=10, showClustermap=showClustermap)
#dft1s.sort_values(by='balanceMarketcapPcnt', ascending=False).loc[:,'pcnt7d'].plot()

In [ ]:
# All Ethereum Tokens PoC
#rmScraperCache()
tokens=True
figsize=50
threshold = 0.5
cmc=CoinMarketCap()
dfbnb = cmc.getAllTokens(tokens=tokens)
#if tokens: dfbnb = dfbnb[dfbnb['quote'] == 'ETH']
#else:      dfbnb = dfbnb[dfbnb['quote'] == 'BTC']
li = list(dfbnb.index)
dft = cmc.getAllTokens(tokens=tokens)
#dft1s = dft[dft['marketCap'] <= 1e6]
dft1s = pm.generatePortfolioT1Supply(dft, balance=(4129.06), risk=1)
dft1s = dft1s.loc[li,:]
sortby = 'riskOn'
#sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
dft1s = dft1s.sort_values(by=sortby, ascending=False)
#none = desc(dft1s)
qg.show_grid(dft1s)
v = DataViz()
v.getAllTokens(tokens=True)
#dft1s = dft1s[(dft1s['vb'] > threshold) & (dft1s['volumePerMarketcap'] > threshold) & (dft1s['riskOn'] > threshold)].sort_values(by='vb', ascending=False)
#dft1s = v.visualizePortfolio(dft1s, li, figsize=300)
v.heatmap(maxx=2500e9, minn=0, usdt=False, figsize=figsize, sortby=sortby, threshold=threshold)
#try: v.heatmap(maxx=1e9, minn=900e6, usdt=False, figsize=15, sortby=sortby, threshold=threshold)
#except: ''
#v.heatmap(maxx=100e6, minn=40e6, usdt=False, figsize=figsize, sortby=sortby, threshold=threshold)
#try: v.heatmap(maxx=1e9, minn=900e6, usdt=False, figsize=15, sortby=sortby, threshold=threshold)
#except: ''
#v.heatmap(maxx=40e6, minn=1e6, usdt=False, figsize=figsize, sortby=sortby, threshold=threshold)
#v.heatmap(maxx=1e6, minn=100e3, usdt=False, figsize=figsize, sortby=sortby, threshold=threshold)
#dft1s.sort_values(by='balanceMarketcapPcnt', ascending=False).loc[:,'pcnt7d'].plot()

In [ ]:
# Yobit PoC
yb = Yobit('4E9AB02C1EBD8D3D6DB6E9C416F6889D', '9ab1dfa0ba89b5738396fbd6f0ee7d6d')
dfy = yb.getCurrencies()
dfbnb = dfy
#dfbnb = dfbnb.head(100)
dfbnb.sort_values(by='base', ascending=True)
#with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000, 'display.max_colwidth', -1):
#    print dfbnb
#print dfbnb['quote']
tokens=True
if tokens: dfbnb = dfbnb[dfbnb['quote'] == 'ETH']
else:      dfbnb = dfbnb[dfbnb['quote'] == 'BTC']
li = list(dfbnb['base'])
#print li
dft = cmc.getAllTokens(tokens=tokens)
#dft1s = dft[dft['marketCap'] <= 1e6]
dft1s = pm.generatePortfolioT1Supply(dft, balance=(5123.83), risk=2)
dft1s = dft1s.loc[li,:]
sortby = 'riskOn'
sortby = 'vb'
#sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
dft1s = dft1s.sort_values(by=sortby, ascending=False)
#none = desc(dft1s)
#qg.show_grid(dft1s)
#dft1s.sort_values(by='balanceMarketcapPcnt', ascending=False).loc[:,'pcnt7d'].plot()
dft1s = dft1s[dft1s['volume'] > 0]
v = DataViz()
dft1s = v.visualizePortfolio(dft1s, li, figsize=300)
qg.show_grid(dft1s.fillna(0), grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

In [ ]:
# yobit test
mdf = p.DataFrame()
sdf = dft1s.head(20)
for i in list(sdf.index):
    mdf = mdf.combine_first(p.DataFrame(apiRequest('https://yobit.net/api/3','/ticker/%s_btc' % i.lower())))
mdf = mdf.transpose()
mdf['spread'] = mdf['sell'] - mdf['buy']
mdf['spreadA'] = mdf['spread'] / mdf['avg'] * 100
mdf.sort_values(by='vol', ascending=False)
Exchange.getBaseQuote()

In [ ]:
# Cryptopia PoC
#"""
res = apiRequest('https://www.cryptopia.co.nz/api','/GetCurrencies')
df = p.DataFrame(res['Data'])
#print df.dtypes
print df.shape
#print df
df.sort_values(by='WithdrawFee', ascending=True)
df = df.set_index('Symbol')#.loc['BTC ETH'.split(' '),:]
print df.shape
df = df.combine_first(dft)
#"""
df

In [ ]:
#df
li = list(df.index)
v = DataViz()
v.portfolioVB(li, show=True, figsize=10)

In [ ]:
%tb

In [ ]:
# run: git pull origin [from the etherdelta source code directory to fetch the latest tokens]
#echo '' > etherdelta.tokens.txt; for i in `ls *ejs | cut -d'.' -f 1`; do echo $i >> etherdelta.tokens.txt; echo $i; done

In [ ]:
print dft1s.head(20)

In [ ]:
# Etherdelta PoC
dft1s = portfolioVBEtherdelta()
qg.show_grid(dft1s, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

In [ ]:
threshold = 0.6
' '.join(list(dft1s[(dft1s['vb'] > threshold) & (dft1s['volumePerMarketcap'] > threshold) & (dft1s['riskOn'] > threshold)].sort_values(by='vb', ascending=False).index))

In [ ]:
def _color_red_or_green(val):
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color

#df.style.applymap(_color_red_or_green)
df.to_html()

In [ ]:
# binance fees
xp = XPath()
url = 'https://support.binance.com/hc/en-us/articles/115000429332'
xresd = {
    'id1' : '//*[@id="article-container"]/article/section[1]/div/div[1]/table/tbody/tr/td[1]/text()',
    'id2' : '//*[@id="article-container"]/article/section[1]/div/div[1]/table/tbody/tr/td[2]/text()',
    'id3' : '//*[@id="article-container"]/article/section[1]/div/div[1]/table/tbody/tr/td[3]/text()',
    'fee' : '//*[@id="article-container"]/article/section[1]/div/div[1]/table/tbody/tr/td[4]/text()',
    'id5' : '//*[@id="article-container"]/article/section[1]/div/div[1]/table/tbody/tr/td[5]/text()',
    'id6' : '//*[@id="article-container"]/article/section[1]/div/div[1]/table/tbody/tr/td[6]/text()',
}
xresd = xp.xpath2df(url, xresd, cache=False)#, verbos
df = p.DataFrame(xresd)
df = df.set_index('id3')
dft = cmc.getAllTokens(tokens=True)
df = dft.combine_first(df)
df = df[df['fee'] > 0]
#df['fee'] = map(lambda x: float(x), df['fee'])
for i in df.index:
    try: df.loc[i,'fee'] = float(df.loc[i,'fee'])
    except: df.loc[i,'fee'] = 0
df['fee'] = p.to_numeric(df['fee'])
df['feeUSD'] = df['fee'] * df['price']
df.sort_values(by='feeUSD', ascending=True)
#dft
#df
"""
li = []
for i in txt.strip().split('\n'):
    spl = i.split('\t')
    for j in range(len(spl)):
        spl[j] = spl[j].strip()
    li.append(spl)
df = p.DataFrame(li[1:], columns=li[0])
df = df.set_index('Short Code')

dft = cmc.getAllTokens(tokens=False)

df = dft.combine_first(df)

df['Fee'] = p.to_numeric(df['Fee'])
df['Confirmations'] = p.to_numeric(df['Confirmations'])
df['feeUSD'] = df['Fee'] * df['price']


df = df[df['feeUSD'] > 0]
df = df[df['volume'] > 0]
df['vf'] = df['volume'] / df['feeUSD']

#df.sort_values(by='feeUSD', ascending=True)#.loc['ETH', :]
df = df.sort_values(by='vf', ascending=True)#.loc['ETH', :]

df

# coinexchange -> binance: base coin showing lowest withdrawal fee
df = df.loc[df2.index, :]
df = df[df['feeUSD'] > 0]
df = df.sort_values(by='feeUSD', ascending=True)#.loc['ETH', :]
df

In [ ]:
# Binance PoC
rmScraperCache()
b = Binance()
v = DataViz()
v.threshold = 0.5
dfbnb = b.getCurrencies()
dfbnb = dfbnb[dfbnb['quote'] == 'BTC']
#dfbnb = dfbnb[dfbnb['quote'] == 'ETH']
li = list(dfbnb['base'])
dft = cmc.getAllTokens(tokens=False)
#dft1s = dft[dft['marketCap'] <= 1e6]
#dft = dft[dft['volume'] >= 1e5]
sortby = 'riskOn'
sortby = 'vb'
#sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
dft1s = pm.generatePortfolioT1Supply(dft, balance=(5148.36), risk=3.32)
dft1s = dft1s.sort_values(by=sortby, ascending=False)
dft1s = v.visualizePortfolio(dft1s, li, figsize=50)
#viewCharts(' '.join(list(v.pdf.index)))
dfp = genHeatmap(list(v.pdf.index))
qg.show_grid(dft1s, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

In [ ]:
flights = sns.load_dataset("flights")
flights['passengers'] = normalizeme(flights['passengers'])
flights['passengers'] = sigmoidme(flights['passengers'])

fd = flights['month'].drop_duplicates()
di = dict(zip(range(1,13), fd.values))
di = dict(zip(di.values(), di.keys()))
flights['m'] = map(lambda x: di[x], flights['month'])
sns.pairplot(flights.loc[:, 'year passengers'.split(' ')], size=10)#, hue=flights['month'])
plt.show()
#sys.exit()
print flights
flights = flights.pivot("month", "year", "passengers")
flights
#flights.plot()
#plt.show()
ax = sns.heatmap(flights)
plt.show()
print flights

In [ ]:
# show charts from poloniex/binance
df = dft1s.sort_values(by=sortby, ascending=False)
list(df['id'])
yy = True
dfs = genDFP(list(df['id']), max=7, logy=yy, normalize=yy, sigmoid=yy)
dfs

In [ ]:
cm['price_usd'].loc[:,'bitcoin'].sort_values(by='price_usd', ascending=False)#['price_usd']

In [ ]:
cm = clustermap(dfs, verbose=False, figsize=20)

In [ ]:
dft = cmc.getAllTokens(tokens=False)
dft = dft[dft['marketCap'] <= 1e6]#.sort_values(by='balanceMarketcapPcnt', ascending=False)
dft.sort_values(by='volumePerMarketcap', ascending=False)

In [ ]:
dft = cmc.getAllTokens(tokens=False)
#dft = cmc.getAllTokens(tokens=True)
#print dft
#cmc.tokens
#dft = dft[dft['marketCap'] > 1e6]
#dft = dft.loc[port.split(),:] #  port coins [port var set below]
df = pm.generatePortfolioT1Supply(dft, balance=(4129.06), risk=1)
ids = list(df['id'])
#print ids
#dfp = genDFP(ids, max=3)
#dfp
#sortby = 'riskOn'
#sortby = 'vb'
sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
df = df.sort_values(by=sortby, ascending=False)#['id']
qg.show_grid(df, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})
#df['id']

In [ ]:
list(df['id'])
dfs = genDFP(list(df['id']), max=20)
dfs

In [ ]:
clustermap(dfs, verbose=True, figsize=15)

In [ ]:
df['t1'] = (df['pcnt1h'] + n.min(df['pcnt1h'])+1) *(df['pcnt24h'] + n.min(df['pcnt24h'])+1) * (df['pcnt7d'] + n.min(df['pcnt7d'])+1) / df['marketCap'] / df['price']
dfs = df.sort_values(by='t1', ascending=False)
dfs.loc[:,'pcnt1h pcnt24h pcnt7d'.split()].plot()
dfs = dfs.loc[:,'marketCap pcnt1h pcnt24h pcnt7d t1'.split()]
dfs = normalizeme(dfs)
dfs = sigmoidme(dfs)
#rcParams['figure.figsize'] = 20, 5
#sns.pairplot(dfs, size=10)
#plt.show()
#dfs
dfs = genDFP(list(dfs['id']), max=10)
dfs

In [ ]:
#df = df.head(300)
#df[fi] = map(lambda x: x.decode('utf-8').strip(), df[fi])
#pp = sns.pairplot(df)#, hue=fi)
#plt.show()

In [ ]:
#port = 'PPP XRL BBT SALT PPT TAAS SIFT PAY OMG KIN AIR PLR LINK RHOC CDT DNT RDN POWR CAT ENG PRO KNC SND AVT HMQ MGO ZRX VERI CVC EOS'
#port = 'XRP STEEM PRO BNB PPP BRAT BCD SALT KIN PLR RHOC PPT CVC CDT NEWB BAS PAY EOS DOM XRL LINK TNT BAT DNT SNT SKIN FUCK NIO HMQ IXT MSP IND DENT FUN BET'

# ib20171128
port = 'RDN AION POWR QSP RHOC DNT LINK ENG FLIXX AVT GMT ZRX CAT HST KNC EOS SALT PRO CVC CREA'

#port = 'CHC QRL BCC XRL RHOC KIN ZRX PPT SALT CDT BRAT PAY MBRS XRL EOS TAAS BNB HMQ OMG WINGS POE CTR LINK SKIN FUCK BAT CVC MCO SNT PPP CAT DOM BBT BET STX TIX DNT MGO VERI ICN NIO BQX TNT ZRX IXT MSP IND FUN LRC PLR'
# eth all > $50
#port = 'CHC RHOC QRL KIN BNB PPT PRO SALT XRP PPP BRAT TAAS PAY CDT XRL DNT'

# coinmarketcap
#port = 'BTC ETH BTH LTC XRP DASH NEM XMR IOTA NEO'

# bitmex
#port = 'BTC BCH DASH ETH ETC LTC XMR XRP XTZ ZEC'

# etoro
port = 'BTC ETH DASH ETC BCH LTC XRP'

# masternode coins
#port = 'CHC DASH NEM PIVX CRW TX EXCL MUE BLOCK BTA ENT SIB BSD DMD KORE BBP'

# git analysis
#port = 'EOS BCH EMC2'

# t1 hoc
#port = 'ETN MIOTA CHC DOGE BTC RDD BURST'

#port = 'BNB ELF'

# portPcnt v [model:t1pi commit:51134db]
#port = 'PPP XRL BBT SNC SALT PPT TAAS EXN SIFT PAY OMG KIN AIR RHOC RDN LINK POWR DNT ENG PRO CAT KNC HMQ AVT ZRX CVC EOS'
# delever::currentPortPcnt v [model:t1pi commit:51134db]
#port = 'BNB RHOC PPT PRO PPP SALT BRAT TAAS CDT XRL PAY DNT KIN HMQ GOOD PLR OMG DOM TIX MBRS CVC SKIN POE WINGS LINK BAS BAT CTR CAT STX FUCK BBT TNT BET ICN NIO EOS VERI NEWB E4ROW BQX IXT MSP IND FUN LRC DENT ZRX'
yy = True
#dfp = genDFP(port, len(port.split(' ')), logy=yy, normalize=yy, sigmoid=yy)
dfp = genHeatmap(port.split(' '))
viewCharts(' '.join(port.split(' ')))
#dfp

In [ ]:
# All Ethereum Tokens PoC
tokens=False
dfbnb = cmc.getAllTokens(tokens=tokens)
#if tokens: dfbnb = dfbnb[dfbnb['quote'] == 'ETH']
#else:      dfbnb = dfbnb[dfbnb['quote'] == 'BTC']
li = list(dfbnb.index)
genHeatmap(li)

In [ ]:
clustermap(dfp, verbose=False, figsize=15)

In [ ]:
dfm = cmc.glo('ethereum')
dfm

In [ ]:
maxx=300e9, minn=10e9,
maxx=10e9, minn=1e9,
maxx=1e9, minn=100e6,
maxx=100e6, minn=10e6,
maxx=10e6, minn=1e6,
maxx=1e6, minn=0,
maxx=1e5, minn=0,

In [ ]:
# Macro Cap [marketcap > 1 billion]
rmScraperCache()
v = DataViz()
v.getAllTokens(tokens=tokens)
#v.getAllTokens(tokens=False)
v.df = v.df[v.df.index != 'USDT']
v.heatmap(maxx=300e9, minn=10e9, usdt=True, figsize=5, sortby=sortby, threshold=threshold)
v.heatmap(maxx=300e9, minn=10e9, usdt=False, figsize=5, sortby=sortby, threshold=threshold)
viewCharts(' '.join(list(v.pdf.index)), showPlot=showPlot)
dfs = genDFP(list(v.pdf['id']), max=7, logy=yy, normalize=yy, sigmoid=yy, showPlot=False)
cm = clustermap(dfs, verbose=False, figsize=10, showClustermap=showClustermap)

In [ ]:
# Macro Cap [marketcap > 1 billion]
v.heatmap(maxx=10e9, minn=1e9, usdt=True, figsize=15, sortby=sortby, threshold=threshold)
v.heatmap(maxx=10e9, minn=1e9, usdt=False, figsize=15, sortby=sortby, threshold=threshold)
viewCharts(' '.join(list(v.pdf.index)), showPlot=showPlot)
dfs = genDFP(list(v.pdf['id']), max=7, logy=yy, normalize=yy, sigmoid=yy, showPlot=False)
cm = clustermap(dfs, verbose=False, figsize=10, showClustermap=showClustermap)

In [ ]:
v = DataViz()
v.getAllTokens(tokens=tokens)
# Mid Cap [1 billion > marketcap > 40 million]
#v.heatmap(maxx=1e9, minn=40e6, usdt=True, figsize=figsize, sortby=sortby, threshold=threshold)
v.heatmap(maxx=1e9, minn=40e6, usdt=False, figsize=figsize, sortby=sortby, threshold=threshold, ascending=True)
#viewCharts(' '.join(list(v.pdf.index)), showPlot=showPlot)
dfs = genDFP(list(v.pdf['id']), max=15, logy=yy, normalize=yy, sigmoid=yy, showPlot=False)
cm = clustermap(dfs, verbose=False, figsize=10, showClustermap=showClustermap)
#qg.show_grid(v.pdf, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})
v.pdf

In [ ]:
# Mid Cap [100 million > marketcap > 10 million]
v.heatmap(maxx=40e6, minn=1e6, usdt=True, figsize=5, sortby=sortby, threshold=threshold)
viewCharts(' '.join(list(v.pdf.index)), showPlot=showPlot)
dfs = genDFP(list(v.pdf['id']), max=7, logy=yy, normalize=yy, sigmoid=yy, showPlot=False)
cm = clustermap(dfs, verbose=False, figsize=10, showClustermap=showClustermap)
v.pdf

In [ ]:
# Mid Cap [10 million > marketcap > 1 million]
v.heatmap(maxx=20e6, minn=100e5, usdt=False, figsize=60, sortby=sortby, threshold=threshold)
viewCharts(' '.join(list(v.pdf.index)), showPlot=showPlot)
dfs = genDFP(list(v.pdf['id']), max=7, logy=yy, normalize=yy, sigmoid=yy, showPlot=False)
cm = clustermap(dfs, verbose=False, figsize=10, showClustermap=showClustermap)
v.pdf

In [ ]:
# Mid Cap [1 million > marketcap > 0]
v.heatmap(maxx=1e6, minn=100e3, usdt=True, figsize=60, sortby=sortby, threshold=threshold)
viewCharts(' '.join(list(v.pdf.index)), showPlot=showPlot)
dfs = genDFP(list(v.pdf['id']), max=7, logy=yy, normalize=yy, sigmoid=yy, showPlot=False)
cm = clustermap(dfs, verbose=False, figsize=10, showClustermap=showClustermap)
v.pdf

In [ ]:
# Mid Cap [1 million > marketcap > 0]
threshold=0.0
v.heatmap(maxx=1e5, minn=0, usdt=True, figsize=10, sortby=sortby, threshold=threshold)
viewCharts(' '.join(list(v.pdf.index)), showPlot=showPlot)
dfs = genDFP(list(v.pdf['id']), max=7, logy=yy, normalize=yy, sigmoid=yy, showPlot=False)
cm = clustermap(dfs, verbose=False, figsize=10, showClustermap=showClustermap)

In [ ]:
# Mid Cap [0 > marketcap > 0]
v.heatmap(maxx=1e6, minn=0, usdt=False, figsize=10, sortby=sortby, threshold=threshold)
viewCharts(' '.join(list(v.pdf.index)), showPlot=showPlot)
dfs = genDFP(list(v.pdf['id']), max=30, logy=yy, normalize=yy, sigmoid=yy, showPlot=False)
cm = clustermap(dfs, verbose=False, figsize=10, showClustermap=showClustermap)

In [ ]:
li = list(df['id'].head(3))
viewCharts(li)

In [ ]:
# show portfolio  charts
port = 'PPP XRL BBT SALT PPT TAAS SIFT PAY OMG KIN AIR PLR LINK RHOC CDT DNT RDN POWR CAT ENG PRO KNC SND AVT HMQ MGO ZRX VERI CVC EOS'
#viewCharts(port)
genHeatmap(port.split(' '))

In [ ]:
# show portfolio eth1 charts
viewCharts('XRP STEEM PRO BNB PPP BRAT BCD SALT KIN PLR RHOC PPT CVC CDT NEWB BAS PAY EOS DOM XRL LINK TNT BAT DNT SNT SKIN FUCK NIO HMQ IXT MSP IND DENT FUN BET')

In [ ]:
# show portfolio eth2 charts
viewCharts('CHC QRL BCC XRL RHOC KIN PPT SALT CDT BRAT PAY MBRS XRL EOS TAAS BNB HMQ OMG WINGS POE CTR LINK SKIN FUCK BAT CVC MCO SNT PPP CAT DOM BBT BET STX TIX DNT MGO VERI ICN NIO BQX TNT ZRX IXT MSP IND FUN LRC PLR')

In [ ]:
nu = 10
for i in list(df.head(nu)['id']):
    cmc.glo(i)
    plt.show()
print '=-=-=-=-=-=-=-=-'
print '=-=-=-=-=-=-=-=-'
for i in list(df.tail(nu)['id']):
    cmc.glo(i)
    plt.show()

In [ ]:
#for i in 'bitcoin ethereum kin'.split(' '):
# b3 == pump n dump example
for i in 'b3coin'.split(' '):
    cmc.glo(i)

In [ ]:
import pandas as p
url = 'https://etherscan.io/txsInternal?a=0xea3f1b409f5e9211eed087c9f5f621d1249b3e7b'
from qore import *
xp = XPath()
xcols = {
    #'to'      : '/html/body/div[1]/div[4]/div[2]/div/div/div/table/tbody/tr/td[6]/span/a/text()',
    'value'   : '/html/body/div[1]/div[4]/div[2]/div/div/div/table/tbody/tr/td[7]/text()',
}
res = xp.xpath2df(url, xcols)
qg.show_grid(p.DataFrame(res))

In [ ]:
#port = 'CHC LBTC BTCRED ARN'
port = 'CHC QRL PPT SALT BNB KIN CDT PRO PPP TAAS'
port = 'PAY GNT PPT QTM LTC'
# portPcnt v [model:t1pi commit:51134db]
#print dft1s.sort_values(by=sortby, ascending=False)['id'].head(10)
#port = ' '.join(list(dft1s.sort_values(by=sortby, ascending=False)['id'].head(10)))
yy = True
dfp = genDFP(port, len(port.split(' ')), logy=yy, normalize=yy, sigmoid=yy)
#dfp

In [ ]:
# dendrogram
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sc

dists = np.array([[0,2,1,4],[2,0,3,5],[1,3,0,6],[4,5,6,0]])
l = ['a','b','c','b']
df = p.DataFrame(dists)
print df#.plot()
Z = sc.linkage(dists, method='complete')
d = sc.dendrogram(Z, labels=l)
label_colors = {'a': 'r', 'b': 'g', 'c': 'm'}
ax = plt.gca()
xlbls = ax.get_xmajorticklabels()
for i in range(len(xlbls)):
    xlbls[i].set_color(label_colors[xlbls[i].get_text()])
plt.show()

In [ ]:
# Load the brain networks example dataset
df = sns.load_dataset("brain_networks", header=[0, 1, 2], index_col=0)

# Select a subset of the networks
used_networks = [1, 5, 6, 7, 8, 12, 13, 17]
used_columns = (df.columns.get_level_values("network")
                          .astype(int)
                          .isin(used_networks))
df = df.loc[:, used_columns]

# Create a categorical palette to identify the networks
network_pal = sns.husl_palette(8, s=.45)
#print network_pal
network_lut = dict(zip(map(str, used_networks), network_pal))

# Convert the palette to vectors that will be drawn on the side of the matrix
networks = df.columns.get_level_values("network")
network_colors = p.Series(networks, index=df.columns).map(network_lut)

# Draw the full plot
sns.clustermap(df.corr(), center=0, cmap="vlag",
               row_colors=network_colors, col_colors=network_colors,
               linewidths=.75, figsize=(13, 13))
#dfd = desc(df)
#df
df#.corr()

In [ ]:
# 0024 mergerTraderSplitter
with p.option_context('display.max_rows', 400, 'display.max_columns', 4000, 'display.width', 1000000):
    print mergerTraderSplitter([13.7, 55.3], '0x0cee9942a8dc4aa6f594d680f6b9654f02ab62d1 0x38a4ff00c207cbd78ab34b6ddd1b8754e4498508'.split(' '), 0.3347784, 'BNB', 'ETH', '0x0CEE9942A8DC4aa6f594d680f6B9654f02Ab62D1') # Binance: BNB -> RDN Etherdelta -> ETH  
    print
    print mergerTraderSplitter([3.65, .7], '0xcdeccaa65fddfa861d322d430a84609213e58678 0xdd4fa0bd3fb7587d6d1af4d4f6270dc2bb85e045'.split(' '), 1418, 'ETH', 'MOD','0xcDECcAa65FDDfA861D322d430A84609213e58678')  # Binance: ETH -> MOD
    print
    print mergerTraderSplitter([228, 1190], '0xdd4fa0bd3fb7587d6d1af4d4f6270dc2bb85e045 0xcdeccaa65fddfa861d322d430a84609213e58678'.split(' '), 473.7625, 'MOD', 'BNB','0xdd4fa0bD3Fb7587D6D1aF4D4f6270Dc2bb85E045')  # Binance: MOD -> BNB
    


In [ ]:
df = p.read_csv('/home/qore/Desktop/binance.TradeHistory.csv')
df.groupby('Price').sum()
df.sort_values('Date', ascending=False)

In [ ]:
df = p.read_csv('/tmp/mdf0.csv', index_col=0)
#print df.dtypes
df = df[ (df['balance'] != 0) | (df['portPcnt'] != 0) ]
print ' '.join(list(df.columns))
li = 'balance balance_eth totalBalanceUsd currentPortPcnt portPcnt balance_usd unitsDiff balanceETHDiff balanceUsdDiff portPcntDiff pcnt7d'.split()
df = df.loc[:, li]
print li
#qg.show_grid(df.loc[:, li], grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})
qg.show_grid(df.loc[:, li], grid_options={'enableColumnReorder':False})

In [ ]:
df = p.read_csv('/tmp/mdf0.csv', index_col=0)
#print df.dtypes
df = df[ (df['balance'] != 0) | (df['portPcnt'] != 0) ]
print ' '.join(list(df.columns))
li = 'balance balance_eth totalBalanceUsd currentPortPcnt portPcnt balance_usd unitsDiff balanceETHDiff balanceUsdDiff portPcntDiff pcnt7d'.split()
df = df.loc[:, li]
print li
#qg.show_grid(df.loc[:, li], grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})
qg.show_grid(df.loc[:, li], grid_options={'enableColumnReorder':False})

In [ ]:
 
xp = XPath()
url = 'http://coinmarketcap.com/currencies/startcoin/#markets'
xresd = {
    'id' : '//div/div[2]/div/table//tr/td[5]/span/text()',
}
xresd = xp.xpath2df(url, xresd, cache=False)#, verbos
print xresd
df = p.DataFrame(xresd)
"""
df = df.set_index('id3')
dft = cmc.getAllTokens(tokens=True)
df = dft.combine_first(df)
df = df[df['fee'] > 0]
#df['fee'] = map(lambda x: float(x), df['fee'])
for i in df.index:
    try: df.loc[i,'fee'] = float(df.loc[i,'fee'])
    except: df.loc[i,'fee'] = 0
df['fee'] = p.to_numeric(df['fee'])
df['feeUSD'] = df['fee'] * df['price']
df.sort_values(by='feeUSD', ascending=True)
#dft
"""
df

In [ ]:
# localCRC.py
# localbtc
# ars to btc
# bitex https://bitex.la/

# https://argenbtc.com/
def process(df):
    df['bidUSD']   = df['bid']   / 17.92
    df['offerUSD'] = df['offer'] / 17.92
    df['spread']     = df['offer'] - df['bid']
    df['spreadPcnt'] = df['offer'] / df['bid'] * 100 - 100
    return df
def t1(name, url, xresd):
    xp = XPath()
    #url = url
    #xresd = {
    #    'bid' : '//*[@id="span_precio_venta_home"]/text()',
    #    'offer' : '//*[@id="span_precio_compra_home"]/text()',
    #}
    xresd = xp.xpath2df(url, xresd, cache=False)#, verbos
    #xresd.update({'url': [url]})
    #print xresd
    #df = p.DataFrame({'argenbtc':xresd}).transpose()
    df = p.DataFrame(xresd, index=[name])#.transpose()
    df['bid']      = p.to_numeric(df['bid'])
    df['offer']    = p.to_numeric(df['offer'])
    df = process(df)
    #print df
    return df

mdf = p.DataFrame()
try:
    mdf = mdf.combine_first(t1('argenbtc', 'https://argenbtc.com', {
        'bid'   : '//*[@id="span_precio_compra_home"]/text()',
        'offer' : '//*[@id="span_precio_venta_home"]/text()',
    }))
except: ''
try:
    mdf = mdf.combine_first(t1('ripio', 'https://www.ripio.com/es/cotizacion-bitcoin/', {
    'bid'   : '/html/body/div[1]/div[1]/div[1]/div[2]/span/text()',
    'offer' : '/html/body/div[1]/div[1]/div[1]/div[1]/span/text()',
    }))
except: ''
try:
    mdf = mdf.combine_first(t1('satoshitango', 'https://www.satoshitango.com/signup', {
    'bid'   : '/html/body/div[1]/div[1]/div/div/div/div[2]/ul[2]/li[1]/a/div/text()',
    #'offer' : '/html/body/div[1]/div[1]/div[1]/div[1]/span/text()',
    }))
except: ''
    
# bitinka
res = apiRequest('https://www.bitinka.pe/api/apinka','/ticker/ARS?format=json')
df = p.DataFrame(res).fillna(0).transpose()
df.loc['ARS','indx'] = 'bitinka'
df['offer'] = df['ask']
df = df.set_index('indx')
df = df.loc[:,'offer bid'.split()]
df = process(df)
mdf = mdf.combine_first(df.loc[:,'bid offer bidUSD offerUSD spread spreadPcnt'.split()])
mdf

In [ ]:
sortby=['pcnt7d', 'pcnt24h','pcnt1h']
sortby='riskOn'
sortby='vb'
threshold = 0.0
showPlot = False
showClustermap = True
yy = True
tokens=True
#tokens=False

In [ ]:
# https://www.bitinka.com.ar/bitinka/api_documentation
res = apiRequest('https://www.bitinka.pe/api/apinka','/volumen24?format=json')
#apiRequest('https://www.bitinka.pe/api/apinka','/ticker?format=json')
#https://www.bitinka.pe/api/apinka/ticker/{MONEDA}?format=json
df = p.DataFrame(res['volumens']).fillna(0)
df

In [ ]:
res = apiRequest('https://www.bitinka.pe/api/apinka','/ticker?format=json')
#https://www.bitinka.pe/api/apinka/ticker/{MONEDA}?format=json
df = p.DataFrame(res).fillna(0).transpose()
df

In [ ]:
# stake
#3000.0/17.92/303.85*839.78#*17.92

# eth1
ci = 130
cF = 1913.62
df = n.array([ci, cF])
df[1] - df[0]
diff = df[1] - df[0]

# eth2
ci = 1300
cF = 7162.038363 + (860+4.13) + 125
df = n.array([ci, cF])
diff = df[1] - df[0]

df = p.DataFrame()
df['ci']   = [130+20,     1300]
df['cF']   = [1913.62,    7162.038363 + (860+4.13) + 125]
df['diff'] = df['cF'] - df['ci']
df['pc'] = df['diff'] / df['ci'] * 100
df['pcntCapital'] = ci * 1 / 100
df['comissionOnProfits'] = df['diff'] * 33.33 / 100
df['netProfits']  = df['diff'] - df['comissionOnProfits']
df['netGross']    = df['ci'] + df['netProfits']
cm = n.sum(df['comissionOnProfits']) - df.loc[0,'comissionOnProfits'] + df.loc[0,'cF']
print cm
print cm * 17.92
df

In [ ]:
# wallet generator
xp = XPath()
url = 'https://walletgenerator.net/'
xresd = {
    'id1' : '/html/body/div[2]/div[6]/div[1]/div[4]/div[2]/div[2]/div/a/text()',
}
xresd = xp.xpath2df(url, xresd, cache=False)#, verbos
df = p.DataFrame(xresd)
df

In [ ]:
icobench
 - https://icobench.com/icos
tokenmarket
 - https://tokenmarket.net/blockchain/all-assets?sorting=best&archived=false&all=true
icodrops
 - https://icodrops.com/category/active-ico/
coingecko
 - https://www.coingecko.com/ico?locale=en
icobazaar
 - https://icobazaar.com/v2/list/featured
cryptocrush
 - https://crushcrypto.com/summary/
coinschedule
 - https://www.coinschedule.com/
 - https://www.coinschedule.com/icos/e1193/stk-pre-sale-and-ico.html
icoalert
 - https://www.icoalert.com/?q=&is_v=1
trackico.io
 - https://www.trackico.io/2/
 - https://www.trackico.io/3/
coincalendar
icotracker
 - https://icotracker.net/
 - https://icotracker.net/current
 - https://icotracker.net/upcoming
 - https://icotracker.net/past

icoprojectsrank
 - https://icoprojectsrank.com/en/
tokenlot 
 - https://www.tokenlot.com/

coinist.io 
listico.io
 - https://www.listico.io/index.php
icorating.com
icowatchlist
hacked.com/ico-analysis
cryptocoinsmarket.com
icohoo.com

stats:
icomarks [ico chart]
 - https://icomarks.com/stats?sort=marketcap-asc
icostats
 - https://icostats.com/

 - https://www.coinist.io/roi/
    
ongoing
 - https://www.coinist.io/ico-listing/
    
cakendars:
coinist
 - https://www.coinist.io/ico-calendar/

In [ ]:
# icos
xp = XPath()
def p1():
    for i in range(1,10):
        url = 'https://icobench.com/icos?page=%s' % i
        xresd = {
            'id1' : '/html/body/div[1]/div/div/div[4]/div/table//tr/td[1]/div[2]/a/text()',
            'href' : '/html/body/div[1]/div/div/div[4]/div/table//tr/td[1]/div[2]/a/@href',
        }
        xresd = xp.xpath2df(url, xresd, cache=False)#, verbos
        #"""
        df = p.DataFrame(xresd)
        print df #"""
        print
        #xresd
        break
p1()

In [1]:
# ethereum pending TXs
res = req.get('https://etherscan.io/chart/pendingtx?output=csv')
res = map(lambda x: x.replace('"', '').replace('\r', '').split(','), res.text.split('\n'))
df = p.DataFrame(res[1:], columns=res[0])
#df = p.read_csv('/home/qore/Desktop/export-PendingQueue.csv')
df = df.set_index('Date(UTC)')
df['Value'] = p.to_numeric(df['Value'])
df.plot()
plt.show()

NameError: name 'req' is not defined

In [52]:
from qore import XPath
import pandas as p
# icos
xp = XPath()
def p1():
    url = 'https://www.numbeo.com/cost-of-living/rankings.jsp'
    xresd = {
        #'id1' : '//table//tr/td[1]/text()',
        'city'                      : '//table//tr/td[2]/text()',
        'costOfLivingIndex'         : '//table//tr/td[3]/text()',
        'rentIndex'                 : '//table//tr/td[4]/text()',
        'costOfLivingPlusRentIndex' : '//table//tr/td[5]/text()',
        'groceriesIndex'            : '//table//tr/td[6]/text()',
        'retaurantIndex'            : '//table//tr/td[7]/text()',
        'localPurchasingPowerIndex' : '//table//tr/td[8]/text()',
    }
    xresd = xp.xpath2df(url, xresd, cache=False)#, verbos
    print xresd.keys()
    """for i in xresd.keys():
        print len(xresd[i])
        print len(xresd[i][len(xresd[i])-511:])
        
        print"""
    for i in xresd.keys():
        xresd[i] = xresd[i][len(xresd[i])-511:]
    """for i in xresd.keys():
        print len(xresd[i])
    for i in xresd.keys():
        print xresd[i]"""
    df = p.DataFrame(xresd)
    for i in xresd.keys():
        try: df[i] = p.to_numeric(df[i])
        except: ''
    #for j in range(2,7):
    #    try: df['id'+str(j)] = p.to_numeric(df['id'+str(j)])
    #    except: ''
    df['y1'] = df['localPurchasingPowerIndex'] / (df['costOfLivingIndex'] * df['groceriesIndex'] * df['rentIndex'] * df['retaurantIndex'])
    with p.option_context('display.max_rows', 600, 'display.max_columns', 4000, 'display.width', 1000000, 'display.max_colwidth', -1):
        #print df.dtypes
        print df.sort_values(by='y1', ascending=False)
    #break
p1()

['retaurantIndex', 'city', 'groceriesIndex', 'rentIndex', 'costOfLivingPlusRentIndex', 'localPurchasingPowerIndex', 'costOfLivingIndex']
                                      city  costOfLivingIndex  costOfLivingPlusRentIndex  groceriesIndex  localPurchasingPowerIndex  rentIndex  retaurantIndex            y1
509  Mangalore, India                       23.17              14.56                      23.88           95.97                      5.68       12.61           2.421652e-03
510  Thiruvananthapuram, India              21.75              13.59                      23.72           62.76                      5.17       12.12           1.941404e-03
497  Vadodara, India                        27.31              15.76                      31.50           67.23                      3.85       16.06           1.263935e-03
506  Bhubaneswar, India                     25.24              15.06                      28.87           52.38                      4.57       14.71           1.069302e-0